# VREF Filter

Objective: Design a Digital (PWM) to Analog filter with fast response time for setting voltage references in motor drives.




In [1]:
using Pkg
Pkg.add("ControlSystems")
Pkg.add("Plots")
Pkg.add("PlotlyBase")
Pkg.add("DifferentialEquations")

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


    Fetching: [======>                                  

    Fetching: [

    Fetching: [=================>                       ]  

    Fetching: [======================

    Fetching: [

Fetching: [========================================>]  99.9 %>   ]  92.0 %

  Resolving package versions...
No Changes to `~/Ultimachine/JupyterNotebooks/VREF_Filter/Project.toml`
No Changes to `~/Ultimachine/JupyterNotebooks/VREF_Filter/Manifest.toml`
  Resolving package versions...
No Changes to `~/Ultimachine/JupyterNotebooks/VREF_Filter/Project.toml`
No Changes to `~/Ultimachine/JupyterNotebooks/VREF_Filter/Manifest.toml`
  Resolving package versions...
No Changes to `~/Ultimachine/JupyterNotebooks/VREF_Filter/Project.toml`
No Changes to `~/Ultimachine/JupyterNotebooks/VREF_Filter/Manifest.toml`
  Resolving package versions...
No Changes to `~/Ultimachine/JupyterNotebooks/VREF_Filter/Project.toml`
No Changes to `~/Ultimachine/JupyterNotebooks/VREF_Filter/Manifest.toml`


In [2]:
using ControlSystems
using Plots
using PlotlyBase
#using DifferentialEquations
plotly(); #nicer plots backend when running live
#gr(); # use GR for publish

┌ Info: Precompiling ControlSystems [a6e380b2-a6ca-5380-bf3e-84a91bcd477e]
└ @ Base loading.jl:1278
┌ Info: Precompiling PlotlyBase [a03496cd-edff-5a9b-9e67-9cda94a718b5]
└ @ Base loading.jl:1278


Construct the transfer function of an RC low pass filter given by:

$$
\frac{1}{1+RCs}
$$

In [3]:
function lowpass_filter(R,C)
    s = tf("s")
    1/(1+R*C*s)
end

lowpass_filter (generic function with 1 method)

In [4]:
24_000_000/2^9

46875.0

In [5]:
"""
Generate a PWM signal given a base clock and resolution (in bits).

clock::Int e.g. 24_000_000
resolution::Int e.g. 8 bit
duty_cycles::Vector{Int} e.g. [0, 128, 255, 64]
"""
function PWM(clock, resolution, values)

    period = resolution/clock
    bitperiod = period/(2^resolution)
    delays = DelayLtiSystem{Float64,Float64}[]
    
    for i in eachindex(values)
        offset = (i-1)*period
        push!(delays, delay(offset)) # initial
        push!(delays, -delay(offset+values[i]*bitperiod)) # turnoff
    end

    return (sum(delays), delays)
end

PWM

In [6]:
# Step function for LTI sim
function u0(out,t)
    if t > 0
        out[1] = 1 #Volts, but doesnt really matter so it is easier to see duty cycle
    else
        out[1] = 0
    end
    return
end

u0 (generic function with 1 method)

In [39]:
# 1.0b filter params

R = 1e2
C = 40e-9

@show 1/(R*C)

f1 = lowpass_filter(R,C)
f2 = lowpass_filter(R,C)
f3 = f1*f2

f4 = lowpass_filter(10e3,1e-9)



1 / (R * C) = 250000.0


TransferFunction{Continuous,ControlSystems.SisoRational{Float64}}
     1.0
--------------
1.0e-5*s + 1.0

Continuous-time transfer function model

In [78]:
sig = cat(map(x->fill(x, 5), trunc.(sin.(0:pi/128:pi/2).*2^9))...,dims=1)

input, input_arr = PWM(27_000_000, 9, sig)#cat(fill((2^9)*1/64, 100) ,fill((2^9)*2/64, 100),fill((2^9)*3/64, 100), fill((2^9)*4/64, 100), dims=1))

@show input_arr

sys = ss(f3)*input

t = 0:2e-8:(input.Tau[end])


@time y, t, x = lsim(sys, u0, t)

LoadError: MethodError: no method matching iterate(::DelayLtiSystem{Float64,Float64})
Closest candidates are:
  iterate(!Matched::SimpleTraits.GenerateTypeVars{:lcase}) at /home/steve/.julia/packages/SimpleTraits/1wYi7/src/SimpleTraits.jl:446
  iterate(!Matched::SimpleTraits.GenerateTypeVars{:lcase}, !Matched::Any) at /home/steve/.julia/packages/SimpleTraits/1wYi7/src/SimpleTraits.jl:446
  iterate(!Matched::ChainRulesCore.One) at /home/steve/.julia/packages/ChainRulesCore/UayCG/src/differentials/one.jl:13
  ...

In [79]:
plot(t, y)

BoundsError: BoundsError: attempt to access 854-element Array{Float64,1} at index [1:5417]

In [13]:
plot(t, y)

<!DOCTYPE html>
 
 
 Plots.jl

In [14]:
bodeplot(f3)

<!DOCTYPE html>
 
 
 Plots.jl

In [15]:
bodeplot(f1)

<!DOCTYPE html>
 
 
 Plots.jl

In [72]:
cat(map(x->fill(x, 10), trunc.(sin.(0:pi/128:pi).*2^9))...,dims=1)

1290-element Array{Float64,1}:
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
 12.0
 12.0
 12.0
  ⋮
 12.0
 12.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0